In [25]:

from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from bs4 import BeautifulSoup
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
import json

In [22]:
import pandas as pd
df = pd.read_csv(r'D:\hackathons\accenture_hackathon\CVAgent\dataset\job_description.csv',encoding='ISO-8859-1')
df = df.iloc[:,:-1]

In [23]:
df.head()

,Job Title,Job Description
0,Software Engineer,Description:\nWe are seeking a skilled Softwa...
1,Data Scientist,Job Description:\nWe are looking for a skilled...
2,Product Manager,Description:\nWe are seeking an innovative and...
3,Cloud Engineer,Description:\nWe are looking for a skilled Clo...
4,Cybersecurity Analyst,Description:\nWe are looking for a skilled Cyb...


In [28]:
df['Job Description'][0]

" Description:\nWe are seeking a skilled Software Engineer to design, develop, and maintain software applications. The ideal candidate will write efficient code, troubleshoot issues, and collaborate with teams to deliver high-quality solutions.\n\nResponsibilities:\n\nDevelop, test, and deploy software applications.\nWrite clean, maintainable, and scalable code.\nCollaborate with cross-functional teams to define and implement features.\nTroubleshoot and debug issues for optimal performance.\nStay updated with emerging technologies and best practices.\nQualifications:\n\nBachelor's degree in Computer Science or a related field.\nProficiency in programming languages like Python, Java, or C++.\nExperience with databases, web development, and software frameworks.\nStrong problem-solving skills and attention to detail.\nAbility to work both independently and in a team environment."

In [ ]:
from langchain_community.embeddings import OllamaEmbeddings
ollama_emb = OllamaEmbeddings(
    model="nomic-embed-text",
)
r1 = ollama_emb.embed_documents(
    [
        "Alpha is the first letter of Greek alphabet",
        "Beta is the second letter of Greek alphabet",
    ]
)
r2 = ollama_emb.embed_query(
    "What is the second letter of Greek alphabet"
)

In [26]:
def readPDF(path):
   # importing required classes
    from pypdf import PdfReader

    # creating a pdf reader object
    reader = PdfReader(path)

    # printing number of pages in pdf file
    print(len(reader.pages))

    # creating a page object
    page = reader.pages[0]

    # extracting text from page
    print(page.extract_text())

In [14]:
import os
from pathlib import Path
directory = "CVAgent_Dataset\CVs1"
for filename in os.listdir(directory):
    readPDF(str(Path(directory+'/'+filename)))


<>:3: SyntaxWarning: invalid escape sequence '\C'
<>:3: SyntaxWarning: invalid escape sequence '\C'
C:\Users\arshan\AppData\Local\Temp\ipykernel_3760\37162023.py:3: SyntaxWarning: invalid escape sequence '\C'
  directory = "CVAgent_Dataset\CVs1"


1
Candidate Resume (ID: C1061)
Name: Alyssa Chavez
Email: alyssachavez88@gmail.com
Phone: +1-465-3587
Education
Diploma in Software Engineering (2013-2015)
Hands-on experience in full-stack web development and mobile app creation.
Work Experience
Data Scientist at ABC Inc. (2019-2023)
Built predictive models that enhanced decision-making processes, reducing operational costs by
25%.
Skills
Cybersecurity - Skilled in penetration testing, risk assessment, and securing enterprise networks
against cyber threats.
Certifications
AWS Certified Solutions Architect - Validated expertise in designing and deploying scalable AWS
solutions, optimizing performance and security.
Achievements
Published a research paper on AI ethics - Contributed to an AI ethics framework adopted by industry
leaders, shaping responsible AI development.
Tech Stack
Java, Spring Boot, MySQL, Kafka, Azure DevOps
2
Candidate Resume (ID: C1070)
Name: Scott Saunders
Email: scottsaunders13@gmail.com
Phone: +1-367-5130
Educatio

In [52]:
instructions="""You will be provided with resumes. Extract and structure information from provided resumes.
            
            resumes: {resume}
            
            Structure the resume provided in following json format:
            {{
                "name": "John Doe",
                "email":"johndoe@email.com",
                "skills": ["Python", "Java", "SQL"],
                "experience": "3 years",
                "education": ["Bachelor's in CS","Diploma in Software Engineering"],
                "certifications": ["AWS Certified Developer"],
                "work_history": ["Software Developer at XYZ for 3 years","Worked at ABC for 1 year"]
            }}
            Provide output in a clear, structured format. Be concise
            If no resume is provided. Return null
            """
instructions_PROMPT= ChatPromptTemplate.from_template(instructions)

In [49]:
import typing
from langchain_community.llms import Ollama



llm = Ollama(model="phi:latest")
llm.invoke("hi")


' Hello there! How can I assist you today? Is there anything specific you would like to know or any queries you have in mind?\n'

In [53]:
chain =     RunnablePassthrough.assign(resume = lambda x: readPDF(x["path"])) | instructions_PROMPT | llm | StrOutputParser()

In [55]:
chain.get_prompts()

[ChatPromptTemplate(input_variables=['resume'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['resume'], input_types={}, partial_variables={}, template='You will be provided with resumes. Extract and structure information from provided resumes.\n            \n            resumes: {resume}\n            \n            Structure the resume provided in following json format:\n            {{\n                "name": "John Doe",\n                "email":"johndoe@email.com",\n                "skills": ["Python", "Java", "SQL"],\n                "experience": "3 years",\n                "education": ["Bachelor\'s in CS","Diploma in Software Engineering"],\n                "certifications": ["AWS Certified Developer"],\n                "work_history": ["Software Developer at XYZ for 3 years","Worked at ABC for 1 year"]\n            }}\n            Provide output in a clear, structured format. Be concise\n            If no resume

In [54]:
chain.invoke({
    "path":r"dataset\CVs1\C1061.pdf"
})

1
Candidate Resume (ID: C1061)
Name: Alyssa Chavez
Email: alyssachavez88@gmail.com
Phone: +1-465-3587
Education
Diploma in Software Engineering (2013-2015)
Hands-on experience in full-stack web development and mobile app creation.
Work Experience
Data Scientist at ABC Inc. (2019-2023)
Built predictive models that enhanced decision-making processes, reducing operational costs by
25%.
Skills
Cybersecurity - Skilled in penetration testing, risk assessment, and securing enterprise networks
against cyber threats.
Certifications
AWS Certified Solutions Architect - Validated expertise in designing and deploying scalable AWS
solutions, optimizing performance and security.
Achievements
Published a research paper on AI ethics - Contributed to an AI ethics framework adopted by industry
leaders, shaping responsible AI development.
Tech Stack
Java, Spring Boot, MySQL, Kafka, Azure DevOps


' {\n    "name": "John Doe",\n    "email": "johndoe@email.com",\n    "skills": ["Python", "Java", "SQL"],\n    "experience": "3 years",\n    "education": [\n        "Bachelor\'s in CS"\n    ],\n    "certifications": [\n        "AWS Certified Developer"\n    ],\n    "work_history": [\n        "Software Developer at XYZ for 3 years",\n        "Worked at ABC for 1 year"\n    ]\n}\n'

In [56]:
pdf_text = readPDF(r"dataset\CVs1\C1061.pdf")

1
Candidate Resume (ID: C1061)
Name: Alyssa Chavez
Email: alyssachavez88@gmail.com
Phone: +1-465-3587
Education
Diploma in Software Engineering (2013-2015)
Hands-on experience in full-stack web development and mobile app creation.
Work Experience
Data Scientist at ABC Inc. (2019-2023)
Built predictive models that enhanced decision-making processes, reducing operational costs by
25%.
Skills
Cybersecurity - Skilled in penetration testing, risk assessment, and securing enterprise networks
against cyber threats.
Certifications
AWS Certified Solutions Architect - Validated expertise in designing and deploying scalable AWS
solutions, optimizing performance and security.
Achievements
Published a research paper on AI ethics - Contributed to an AI ethics framework adopted by industry
leaders, shaping responsible AI development.
Tech Stack
Java, Spring Boot, MySQL, Kafka, Azure DevOps


In [60]:
from langchain.chat_models import ChatOllama
from langchain.schema import SystemMessage, HumanMessage

# Load Ollama Model
llm = ChatOllama(model="phi")

# Define prompt for structuring CV data
prompt = f"""
Extract the following details from this CV:
- Name
- Skills (as a list)
- Experience (years)
- Education
- Certifications (as a list)
- Work History (as a list)

Return the result as a JSON object.

CV Text:
{pdf_text}
"""

# Get structured response
messages = [SystemMessage(content="Extract structured data from a CV."), HumanMessage(content=prompt)]
response = llm(messages)
print(response.content)  # JSON output


 import json

def extract_details(cv):
    data = {
        "name": cv["name"],
        "skills": cv["skills"],
        "experience": cv["years"],
        "education": cv["education"],
        "certifications": cv["certifications"],
        "work_history": cv["work_history"]
    }

    return json.dumps(data)

cv = {
    "name": "Jane Doe",
    "skills": ["Python", "JavaScript", "CSS"],
    "years": 5,
    "education": [
        {
            "degree": "Bachelor's Degree in Computer Science",
            "year": 2017
        },
        {
            "degree": "Master's Degree in Data Analysis",
            "year": 2019
        }
    ],
    "certifications": [
        {
            "name": "Python Certification",
            "expiration_date": "2021-06-30"
        },
        {
            "name": "JavaScript Certification",
            "expiration_date": "2022-01-31"
        }
    ],
    "work_history": [
        {
            "company": "ABC Inc.",
            "position": "Software Dev